In [2]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import re
import gensim
from gensim.models import KeyedVectors
import numpy as np
from collections import Counter
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases
from nltk import sent_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import gensim.downloader as api
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, make_scorer
from preprocessing import *

In [122]:
dataset = pd.read_csv("amazon_alexa.tsv", sep = "\t")
dataset.drop(dataset[dataset.rating == 3].index, inplace=True) #droppa recensioni con 3 stelle
# dataset.drop(dataset[dataset.rating == 4].index, inplace=True) #droppa recensioni con 4 stelle
print(dataset.shape)
dataset.head()

(2998, 5)


,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [123]:
dataset = drop_positive_duplicates(dataset)
dataset.shape

(2453, 5)

In [13]:
new_text, new_sent_tok = tokenize_list_of_text(dataset["verified_reviews"], custom_stopwords, False, pos_list)
# contengono una lista di tutte le frasi pre processate, nella prima variabile in stringa, nella seconda in tokens 

total number of types extracted is: 3174


In [14]:
negative_reviews = list(dataset[dataset["feedback"] == 0]["verified_reviews"].values)
negative_text, negative_tok = tokenize_list_of_text(negative_reviews, custom_stopwords, False, pos_list)

total number of types extracted is: 1240


In [15]:
negative_artifical = generate_samples(negative_tok, 150)
new_sent_tok.extend(negative_artifical)
len(negative_artifical)

141

In [17]:
def frequency_cleaning(new_sent_tok, n):

    tot_tokens = []

    for sent in new_sent_tok:
        for tok in sent:
            tot_tokens.append(tok)

    freqs = nltk.FreqDist(tot_tokens)
    cleaned_reviews = []

    for sent in new_sent_tok:
        clean_sent = []
        for tok in sent:
            if freqs[tok] > n:
                clean_sent.append(tok)
        cleaned_reviews.append(clean_sent)

    return cleaned_reviews

In [18]:
# Elimino token con freq minore di 5
cleaned_reviews = frequency_cleaning(new_sent_tok, 4)
negative_artificial_cleaned = cleaned_reviews[-len(negative_artifical):] # estraggo le recensioni artificiali per poi aggiungerle esclusivamente al train
del cleaned_reviews[-len(negative_artifical):] # le elimino dalle recensioni pulite

In [31]:
bigrams = Phrases(cleaned_reviews, scoring="npmi", threshold=0) #estrae le collocazioni tramite PMI
bigrams[cleaned_reviews][0]

['like_fact',
 'answer',
 "n't_see",
 'real',
 'need',
 'household',
 'though',
 'good',
 'bargain',
 'day',
 'deal']

In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(bigrams[cleaned_reviews], list(dataset["feedback"].values), test_size=0.20, random_state=10)
X_train.extend(negative_artificial_cleaned)
Y_train.extend([0 for x in range(len(negative_artifical))])

In [79]:
# Questa parte estrare le parole più rilevanti per ogni categoria, ma non la uso alla fine.
# Lo riprenderò più avanti
# FARE IL K-SQUARED

neg_tokenized = [rev for rev, feedback in zip(X_train, Y_train) if feedback == 0]
pos_tokenized = [rev for rev, feedback in zip(X_train, Y_train) if feedback == 1]
neg_tokenized = [w for rev in neg_tokenized for w in rev]
pos_tokenized = [w for rev in pos_tokenized for w in rev]

import math 

def get_pos_neg_score(pos, neg):


    total_tokens = pos + neg
    total_tokens_freq = nltk.FreqDist(total_tokens)
    pos_freq = nltk.FreqDist(pos)
    neg_freq = nltk.FreqDist(neg)
    data_dict = {"pos": {}, "neg": {}}

    for w in pos:
        p_category = pos_freq[w] / total_tokens_freq[w]
        p = pos_freq[w] / len(pos)
        h = -math.log2(p)
        data_dict["pos"][w] = p_category * p
    
    for w in neg:
        p_category = neg_freq[w] / total_tokens_freq[w]
        p = neg_freq[w] / len(neg)
        h = -math.log2(p)
        data_dict["neg"][w] = p_category * p 

    pos_list = list(data_dict["pos"].values())
    neg_list = list(data_dict["neg"].values())
    scaled_pos = [(float(i)-min(pos_list))/(max(pos_list)-min(pos_list)) for i in pos_list]
    scaled_neg = [(float(i)-min(neg_list))/(max(neg_list)-min(neg_list)) for i in neg_list]

    data_dict["pos"] = {k: v for k, v in zip(data_dict["pos"], scaled_pos)}
    data_dict["neg"] = {k: v for k, v in zip(data_dict["neg"], scaled_neg)}
    
    pos_sorted = {k: v for k, v in sorted(data_dict["pos"].items(), key = lambda item: item[1], reverse=True)}
    neg_sorted = {k: v for k, v in sorted(data_dict["neg"].items(), key = lambda item: item[1], reverse=True)}
    data_dict["pos"] = pos_sorted
    data_dict["neg"] = neg_sorted

    risultato = {}

    for w in total_tokens:
        pos_value = data_dict["pos"].get(w, 0)
        neg_value = data_dict["neg"].get(w, 0)
        risultato[w] = max(pos_value, neg_value)

    return {k: v for k, v in sorted(risultato.items(), key = lambda item: item[1], reverse=True)}

data_dict = get_pos_neg_score(pos_tokenized, neg_tokenized)

In [80]:
data_dict

{'love': 1.0,
 'device': 1.0,
 'run': 0.9079009034705069,
 'would': 0.8453037956062937,
 'product': 0.7014903579510192,
 'month': 0.5653372573717017,
 'return': 0.5416020734424959,
 'become': 0.48289814587403274,
 'use': 0.45370052049793513,
 'non': 0.4491203883129327,
 'contain': 0.44858581209831094,
 'prove': 0.4346178529420651,
 'function': 0.4325485256596583,
 'great': 0.43136777942824506,
 'not': 0.4061915940343946,
 'matter': 0.38924785227529635,
 'charge': 0.38924785227529635,
 'care': 0.3867277072635081,
 'disconnect': 0.38505348505287956,
 'work': 0.38489487452766213,
 'music': 0.35417478212535053,
 'yield': 0.3523620934663954,
 'try': 0.33477281156593774,
 'clock': 0.32917304415645177,
 'money': 0.32429684219875343,
 'like': 0.3206433633341013,
 'sound': 0.3158486288069632,
 'lack': 0.31375381129926877,
 'time': 0.31022274082414275,
 'point': 0.3100236572683526,
 'second': 0.3060726530639209,
 'half': 0.3050139144570402,
 'get': 0.30414941822563274,
 'work_great': 0.291712102

In [33]:
# tfidf weighting sui token, per ora senza pos. Mi sono reso conto che la pos per averla più accurata la devo fare prima del pre processing.

tfidf = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False, min_df = 0)
tfidf_model = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)
review = 12
for score, feature in zip(tfidf_model.toarray()[review], tfidf.get_feature_names_out()):
    if score > 0.0:
        print(feature, score)

able 0.2506518360082596
bedroom 0.24149617460044245
completely 0.32141403445257694
dark 0.31565380416962274
display 0.27911096078563424
easy_use 0.23804259194392918
favorite 0.27660244004665774
love_spot 0.2845504985597484
music 0.1793845855811626
night 0.2506518360082596
skill 0.247856479066645
try 0.2253828518169239
turn 0.22624845728748494
wake 0.2567729936294362
yet 0.24394195584094175


In [34]:
w2v_model = Word2Vec(X_train, vector_size=300, window = 5, min_count = 0, sg=1, hs = 1, alpha=0.03, min_alpha=0.0007, seed = 5)

In [35]:
w2v_model.train(X_train, total_examples=len(X_train), epochs=100)

(1953622, 2152500)

In [36]:
w2v_model.wv.most_similar("love", topn = 10)

[('bought', 0.22972042858600616),
 ('use_voice', 0.2281317561864853),
 ('able_watch', 0.20024116337299347),
 ('plus_great', 0.19483120739459991),
 ('smart_device', 0.19477352499961853),
 ('movie', 0.18927587568759918),
 ('longer', 0.18595993518829346),
 ('music_video', 0.18573375046253204),
 ('dad', 0.18504254519939423),
 ('like_new', 0.1827973872423172)]

In [37]:
w2v_model.wv.most_similar("love", topn = 2000)[-10:]

[('guarantee', -0.045008983463048935),
 ('attempt', -0.045589104294776917),
 ('glitch', -0.046338461339473724),
 ('unnecessary', -0.05380210280418396),
 ('around', -0.05682726204395294),
 ('sell_refurbished', -0.05845823138952255),
 ('clear', -0.06318702548742294),
 ('call', -0.06369436532258987),
 ('charge', -0.06837613135576248),
 ('loose', -0.08004898577928543)]

In [38]:
df = pd.DataFrame(tfidf_model.toarray(), columns = tfidf.get_feature_names_out())
df

,'ll,'re,'ve,'ve_use,....,.....,......,ability,able,able_see,...,yard,year,year_old,yell,yes,yet,yield,z-wave,zero,zigbee
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.290287,0.0
2100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [39]:
g_model = []
# google_model = api.load('word2vec-google-news-300', True)
# g_model = KeyedVectors.load_word2vec_format(google_model, binary=True)

In [108]:
def review_vectors(tokens, size = 300, weights = [], pretrained = False):

    """Genera un vettore per ogni recensione: questo vettore
    è calcolato come la media ponderata (t * w: token vettore * peso tfidf) dei vettori dei token nella recensione.
    Il vettore risultante è normalizzato alla fine."""

    vec = np.zeros(size).reshape((1, size))
    count = 0

    if len(tokens) != len(weights):
        print("nope")

    for word, weight in zip(tokens, weights):
        try:
            if pretrained:
                vec += g_model[word] * weight
            else:
                vec += w2v_model.wv[word] * weight 
            count +=1
        except KeyError:
            print("non trovo", word)
            continue

    if count!= 0:
        vec = vec / norm(vec)
        
    return vec

In [109]:
# qui viene creato per ogni recensione il vettore

w2v_X_train = np.zeros((len(X_train), 300))
for i in range(len(X_train)):
    w2v_X_train[i,:] = review_vectors(tfidf.inverse_transform(tfidf_model[i, :])[0], 300, tfidf_model[i,:].data, False)
w2v_df = pd.DataFrame(w2v_X_train)
w2v_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.062063,-0.062618,0.058478,0.014673,0.071422,-0.024450,-0.018914,-0.031099,0.110349,-0.026182,...,0.077885,-0.041908,-0.040014,0.024018,0.015754,-0.044350,-0.020905,0.061656,0.046637,-0.022577
1,0.006357,-0.197008,0.011488,0.100086,-0.021176,-0.003958,0.021541,-0.184806,0.023250,0.003977,...,-0.056645,-0.085147,-0.021859,0.053982,-0.007105,0.013391,-0.068238,0.013340,0.030610,-0.010731
2,-0.062063,-0.062618,0.058478,0.014673,0.071422,-0.024450,-0.018914,-0.031099,0.110349,-0.026182,...,0.077885,-0.041908,-0.040014,0.024018,0.015754,-0.044350,-0.020905,0.061656,0.046637,-0.022577
3,-0.046856,-0.058209,-0.007275,0.050751,-0.061454,0.043961,-0.055007,0.032040,-0.029792,0.041518,...,-0.017711,-0.053345,-0.025614,0.047275,0.027194,0.029139,-0.083463,0.048183,0.096344,-0.025824
4,0.000375,-0.020686,-0.122444,-0.034106,-0.016997,-0.023606,0.018873,-0.013750,0.016218,-0.056624,...,-0.033082,-0.110452,-0.049898,0.104750,0.116378,-0.061139,-0.022526,0.016022,0.000831,-0.019158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,-0.030754,-0.038204,-0.046718,-0.051567,-0.112098,-0.064093,-0.072110,-0.006339,0.012298,-0.019871,...,-0.011240,-0.014607,-0.099375,0.068001,0.035117,0.039829,-0.075604,0.044743,-0.001160,-0.039497
2099,-0.040262,-0.078321,-0.008823,0.055680,-0.031582,-0.037696,-0.149164,-0.057952,0.004249,0.020905,...,-0.018158,0.035344,-0.044336,0.050847,0.044671,-0.024443,-0.154307,0.024719,0.043729,-0.004214
2100,-0.038655,-0.054145,-0.036748,0.064375,-0.039291,-0.060229,0.016708,-0.164744,-0.010326,0.010940,...,0.010938,-0.036528,-0.007417,0.037110,-0.015500,0.096834,-0.082517,0.112574,0.017908,0.002096
2101,-0.030610,-0.099081,0.074273,0.011618,0.002749,0.010126,-0.078924,-0.056787,-0.014685,0.055787,...,-0.033128,-0.080416,-0.050150,-0.030996,0.128987,0.039046,-0.101774,0.087411,0.038001,-0.035037


In [110]:
X_test_vec = np.zeros((len(X_test), 300))
for i in range(len(X_test)):
    X_test_vec[i,:] = review_vectors(tfidf.inverse_transform(X_test_tf[i, :])[0], 300, X_test_tf[i,:].data, False)

X_test_vec[0]

array([-5.81027974e-02, -1.55701393e-02, -6.65868345e-03,  6.70399874e-03,
       -4.72705774e-02,  1.97437448e-04, -9.20691091e-02, -4.54569530e-02,
       -1.37483326e-02, -6.14612290e-02, -2.75654764e-03,  3.62611553e-02,
        8.54498556e-02, -3.85121775e-02, -8.93467703e-03,  2.13747139e-02,
        6.51300994e-02, -2.90841430e-02, -9.29665557e-03, -6.77522429e-02,
        5.69960670e-02, -2.78584610e-02,  2.75656784e-02,  1.72046428e-02,
       -6.41857991e-02,  2.63469703e-02, -6.63981070e-02,  6.06875847e-03,
       -9.69497173e-02, -2.72950427e-02, -6.57164772e-02,  4.86553661e-02,
       -7.93267958e-02, -4.83421310e-02, -3.67140941e-02, -2.58310649e-02,
       -1.83663561e-02,  2.38916872e-02, -2.67583258e-02,  1.31817417e-01,
       -1.40228888e-02, -1.06277737e-01,  2.82544221e-02, -3.69687330e-02,
        5.65514458e-02,  3.26416123e-02,  6.61670927e-02,  7.51988367e-02,
        4.76062621e-02,  3.53623317e-03,  1.33446965e-02,  9.72120969e-02,
       -9.76809422e-03,  

In [111]:
svm = SVC(max_iter=3000, class_weight = "balanced", C = 100, gamma = 1, kernel = "rbf")
svm_model = svm.fit(w2v_X_train, Y_train)
predictions = svm_model.predict(X_test_vec)
print(metrics.classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.83      0.92      0.87        85
           1       0.98      0.96      0.97       406

    accuracy                           0.95       491
   macro avg       0.91      0.94      0.92       491
weighted avg       0.96      0.95      0.95       491



In [102]:
# risultati migliori:

# w2v + tf-idf weights + pos/neg score + svm and word count > 4

# w2v configuration:
# w2v_model = Word2Vec(cleaned_reviews, vector_size=300, window = 5, min_count = 0, sg=1, hs = 1, alpha=0.03, min_alpha=0.0007) and 100 epochs for training

# otherwise the balanced dataset performs better

In [103]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [104]:
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf','linear', 'poly'],
              'class_weight':['balanced', None]
}

In [105]:
grid = GridSearchCV(SVC(), param_grid, refit = True, scoring = "f1_macro", cv=kfold, verbose=3)
grid.fit(w2v_X_train, Y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.867 total time=   0.8s
[CV 2/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.860 total time=   0.8s
[CV 3/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.863 total time=   0.8s
[CV 4/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.885 total time=   0.8s
[CV 5/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.884 total time=   0.8s
[CV 1/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.835 total time=   0.4s
[CV 2/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.843 total time=   0.4s
[CV 3/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.836 total time=   0.4s
[CV 4/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.865 total time=   0.4s
[CV 5/5] END C=0.1, class_weight=balanced, gamma=1, kernel=lin

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'class_weight': ['balanced', None],
                         'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'linear', 'poly']},
             scoring='f1_macro', verbose=3)

In [106]:
print(grid.best_params_)

{'C': 10, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'rbf'}


In [107]:
grid_predictions = grid.predict(X_test_vec)

print(classification_report(Y_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.87      0.92      0.89        85
           1       0.98      0.97      0.98       406

    accuracy                           0.96       491
   macro avg       0.92      0.94      0.93       491
weighted avg       0.96      0.96      0.96       491

